# MODISの8日合成LSTデータをエクスポートする

使用データ：
- MODIS MOD11A2（LST Day 1km）
- 期間：2000年2月18日から2025年7月20日
- 解像度：1km
- 観測周期：8日合成
- リンク： https://developers.google.com/earth-engine/datasets/catalog/MODIS_061_MOD11A2

対象領域：
- ベトナム　ハノイ

作成元pythonファイル：
LST_time_series_analysis.py

In [1]:
# ライブラリのインポート
import ee
import geemap
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import japanize_matplotlib
import calendar
import numpy as np
import datetime

# Google Earth Engineの認証

In [2]:
GGE_PROJECT = 'master-research-465403'  # Google Earth EngineプロジェクトID
# Google Earth Engine APIの初期化
try:
       ee.Initialize(project=GGE_PROJECT)
except Exception as e:
       ee.Authenticate()
       ee.Initialize(project=GGE_PROJECT)

# 対象領域の設定

In [3]:
# SHPファイルの読み込み
shp_path = '../data/SHP/研究対象領域/研究対象都市_行政区画.shp'
# 行政区画データの読み込み
boundary_df = gpd.read_file(shp_path)

hanoi_row = boundary_df[boundary_df['TinhThanh'] == 'Hà Nội'].iloc[0]
hanoi_geometry = hanoi_row.geometry
hanoi_geojson = hanoi_geometry.__geo_interface__

# ROIの設定
ROI = ee.Geometry(hanoi_geojson)

# 可視化
Map = geemap.Map(center=[21.0285, 105.8542], zoom=10)
Map.addLayer(ROI, {}, 'Hanoi ROI')
Map


Map(center=[21.0285, 105.8542], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topr…

# 衛星データの取得期間の設定および取得

## MODIS MOD11A2データの概要
URL: https://developers.google.com/earth-engine/datasets/catalog/MODIS_061_MOD11A2#bands

<details>
<summary>MODIS MOD11A2のバンド情報</summary>

#### 主なバンドと意味

- **LST_Day_1km / LST_Night_1km**
  - 地表面温度（昼/夜）バンド。単位はケルビン（K）。
  - 値の範囲は 7500～65535。  
  - 実際の温度値は「値 × 0.02」でケルビンに変換。摂氏にするには「(値 × 0.02) - 273.15」。

- **QC_Day / QC_Night**
  - 品質管理（Quality Control）バンド。各ピクセルの品質情報をビットマスクで格納。
  - 主なビットの意味：
    - **Bits 0-1（必須QAフラグ）**
      - 0: 良好な品質（詳細QA不要）
      - 1: 信頼性低い（詳細QA推奨）
      - 2: 雲の影響で未取得
      - 3: その他理由で未取得
    - **Bits 2-3（データ品質）**
      - 0: 良好
      - 1: その他
    - **Bits 4-5（放射率誤差）**
      - 0: ≤0.01
      - 1: ≤0.02
      - 2: ≤0.04
      - 3: >0.04
    - **Bits 6-7（LST誤差）**
      - 0: ≤1K
      - 1: ≤2K
      - 2: ≤3K
      - 3: >3K

- **Day_view_time / Night_view_time**
  - 観測時刻（ローカル時間）。単位は時間（h）、値は0～24。

- **Day_view_angl / Night_view_angl**
  - 観測時のセンサ視野角。単位は度（deg）。

- **Emis_31 / Emis_32**
  - バンド31/32の放射率。値 × 0.002 + 0.49 で実値。

- **Clear_sky_days / Clear_sky_nights**
  - 8日間の各日（昼/夜）が「晴天」だったかをビットで記録。
  - 例：Bit 0が1なら1日目は晴天、0なら曇天。

#### 実務でのポイント

- **LST値の変換**  
  画像値 × 0.02 → ケルビン。摂氏はさらに -273.15。
- **品質管理（QC）**  
  雲や誤差の大きいピクセルはQCバンドで除外可能。
- **晴天日判定**  
  8日間のうち晴天だった日数も取得できる。

</details>

##  プログラムの解説
<details>
<summary>プログラムの解説</summary>

mask = qc.bitwiseAnd(3).eq(0) 
mask = mask.And(qc.rightShift(6).bitwiseAnd(3).lte(1))


- bitwiseAnd(3) →　下位2ビットを抽出　10進数3 は 2進数で11
例： QC_Day: 10101010　の場合

｜QC_Day:   10101010 ｜　　
｜3:        00000011 ｜　　
｜AND    :  00000010 ｜　　

AND演算は、両方のビットが1のときだけ1になります。
つまり、下位2ビット（右端2つ）だけが残り、それ以外は0になるという仕組みです。

- rightShift(6)

6ビット右にシフトすることで、QC_Dayのbits 6-7（LST誤差フラグ）を下位2ビットに移動します。
その後、bitwiseAnd(3)でbits 6-7だけを抽出します。

- eq(0), lte(1), gt(1)

eq(0)は「値が0と等しい」ピクセルを抽出します。
lte(1)は「値が1以下」のピクセルを抽出します。
gt(1)は「値が1より大きい」ピクセルを抽出します。

</details>

In [ ]:
# 期間の設定
YEAR = 2023  # 2001 ~2024年までのデータを取得
quality_stats = []

# 各月の平均LSTを算出する
for MONTH in range(1, 13):
    START_DATE = f"{YEAR}-{MONTH:02d}-01"
    END_DAY = calendar.monthrange(YEAR, MONTH)[1]
    END_DATE = f"{YEAR}-{MONTH:02d}-{END_DAY:02d}"

    def add_quality_mask(img):
        qc = img.select('QC_Day')
        mask = qc.bitwiseAnd(3).eq(0)  # bits 0–1 == 00
        lst = img.select('LST_Day_1km').multiply(0.02).subtract(273.15).rename('LST_Celsius')
        return img.addBands(lst).updateMask(lst.mask()).set('high_quality_mask', mask)

    dataset = ee.ImageCollection('MODIS/061/MOD11A2') \
        .filterDate(START_DATE, END_DATE) \
        .filterBounds(ROI) \
        .map(add_quality_mask)

    # 各画像ごとに品質統計を取得
    for img in dataset.toList(dataset.size()).getInfo():
        img_id = ee.Image(img['id'])
        date_str = ee.Date(img['properties']['system:time_start']).format('YYYY-MM-dd').getInfo()

        # 高品質ピクセルのマスクと全ピクセル数をカウント
        qc = img_id.select('QC_Day')
        high_quality_mask = qc.bitwiseAnd(3).eq(0)
        total_mask = qc.mask()

        # カウント計算
        high_quality_count = high_quality_mask.reduceRegion(
            reducer=ee.Reducer.sum(),
            geometry=ROI,
            scale=1000,
            maxPixels=1e13
        ).getInfo()['QC_Day']

        total_count = total_mask.reduceRegion(
            reducer=ee.Reducer.count(),
            geometry=ROI,
            scale=1000,
            maxPixels=1e13
        ).getInfo()['QC_Day']

        ratio = high_quality_count / total_count if total_count > 0 else None

          # 観測時刻（Day_view_time）のROI平均値
        day_view_time_mean = img_id.select('Day_view_time').reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=ROI,
            scale=1000,
            maxPixels=1e13
        ).getInfo().get('Day_view_time', None)
        # 0.1時間単位なので10で割る
        if day_view_time_mean is not None:
            day_view_time_mean = float(day_view_time_mean) * 0.1

        quality_stats.append([date_str, high_quality_count, total_count, ratio, day_view_time_mean])

    # 月平均LSTのエクスポート
    monthly_mean = dataset.select('LST_Celsius').mean()
    geemap.ee_export_image(
        monthly_mean,
        filename=f'LST_{YEAR}_{MONTH:02d}.tif',
        scale=1000,
        region=ROI,
        file_per_band=False
    )

# 結果をCSV出力
df = pd.DataFrame(quality_stats, columns=['観測日', '高品質ピクセル数', '全ピクセル数', '比率','観測時刻(平均,時)'])
df.to_csv(f'{YEAR}年の品質統計.csv', index=False)




Generating URL ...
Please wait ...
Data downloaded to /workspaces/master-s_research_env/workspace/note/LST_2001_01.tif
Generating URL ...
Please wait ...
Data downloaded to /workspaces/master-s_research_env/workspace/note/LST_2001_02.tif
Generating URL ...
Please wait ...
Data downloaded to /workspaces/master-s_research_env/workspace/note/LST_2001_03.tif
Generating URL ...
Please wait ...
Data downloaded to /workspaces/master-s_research_env/workspace/note/LST_2001_04.tif
Generating URL ...
Please wait ...
Data downloaded to /workspaces/master-s_research_env/workspace/note/LST_2001_05.tif
Generating URL ...
Please wait ...
Data downloaded to /workspaces/master-s_research_env/workspace/note/LST_2001_06.tif
Generating URL ...
Please wait ...
Data downloaded to /workspaces/master-s_research_env/workspace/note/LST_2001_07.tif
Generating URL ...
Please wait ...
Data downloaded to /workspaces/master-s_research_env/workspace/note/LST_2001_08.tif
Generating URL ...
Please wait ...
Data download